<a href="https://colab.research.google.com/github/josemoti1999/data_analysis_argoid/blob/master/argoid_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2


###Introduction
In this notebook we will predict the items bought together with a given item. The final dictionary is saved as **'answer_dict.pickle'**. The itemcodes to label description conversion is done with **'label_dict.pickle'**. See Answers display section for final answers.

Give the link to the folder here remaining part is unchanged.

In [1]:
cd /content/drive/My Drive/Kaggle

/content/drive/My Drive/Kaggle


In [0]:
import pandas as pd

In [5]:
df_final=pd.read_csv('cleaned_data.csv')
df_final.shape

(534062, 8)

In [0]:
df_final = df_final[df_final['ItemCode']!=-1]

In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531286 entries, 0 to 534061
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   UserId                  531286 non-null  float64
 1   TransactionId           531286 non-null  int64  
 2   TransactionTime         531286 non-null  object 
 3   ItemCode                531286 non-null  int64  
 4   ItemDescription         531286 non-null  object 
 5   NumberOfItemsPurchased  531286 non-null  int64  
 6   CostPerItem             531286 non-null  float64
 7   Country                 531286 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 36.5+ MB


In [8]:
item_id_array = df_final['ItemCode'].unique()
item_id_array.shape

(3305,)

In [9]:
item_description_array = df_final['ItemDescription'].unique()
item_description_array.shape

(4030,)

In [0]:
df_final['TotalCost'] = df_final['NumberOfItemsPurchased']*df_final['CostPerItem']

In [0]:
#grouping by TransactionId

grouped = df_final.groupby('TransactionId')
temp = pd.DataFrame()
temp['UserId'] = grouped['UserId'].unique()
temp['TransactionTime'] = grouped['TransactionTime'].unique()
temp['ItemCode'] = grouped['ItemCode'].unique()
temp['Country'] = grouped['Country'].unique()
temp['TotalCost'] = grouped['TotalCost'].sum()

temp['UserId'] = temp['UserId'].apply(lambda x:x[0])
temp['TransactionTime'] = temp['TransactionTime'].apply(lambda x:x[0])
temp['Country'] = temp['Country'].apply(lambda x:x[0])

In [12]:
temp.sort_index(axis=0)
temp.reset_index(inplace=True)
temp.head()

,TransactionId,UserId,TransactionTime,ItemCode,Country,TotalCost
0,5900015,374850.0,Mon Feb 12 04:26:00 IST 2018,"[477792, 1492113, 1764609, 1772526, 1787583, 4...",United Kingdom,576.30
1,5900026,374850.0,Mon Feb 12 04:28:00 IST 2018,"[475272, 475293]",United Kingdom,92.16
2,5900037,273987.0,Mon Feb 12 04:34:00 IST 2018,"[1011927, 456834, 456855, 1782459, 477729, 477...",United Kingdom,1155.54
3,5900048,273987.0,Mon Feb 12 04:34:00 IST 2018,"[481194, 481173, 481152, 482160]",United Kingdom,290.34
4,5900059,273987.0,Mon Feb 12 04:35:00 IST 2018,[456876],United Kingdom,73.98


In [0]:
df_final=temp
df_final['TransactionTime'] = df_final['TransactionTime'].apply(lambda x:x.replace('2028', '2018'))

In [14]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23198 entries, 0 to 23197
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionId    23198 non-null  int64  
 1   UserId           23198 non-null  float64
 2   TransactionTime  23198 non-null  object 
 3   ItemCode         23198 non-null  object 
 4   Country          23198 non-null  object 
 5   TotalCost        23198 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


###Gettting dictionary of answers

In [23]:
from collections import Counter
from itertools import combinations

count=Counter()
for row in df_final['ItemCode']:
    l1 = [int(i) for i in row]
    l1.sort()
    count.update(Counter(combinations(l1,2)))
print(count.most_common(10))

[((470106, 1787079), 967), ((460551, 1787079), 876), ((470631, 1787079), 836), ((476637, 476679), 783), ((435225, 1787079), 706), ((487263, 1787079), 705), ((487179, 1787079), 669), ((435225, 470043), 663), ((435225, 435267), 648), ((434952, 1787079), 647)]


In [26]:
len(count.most_common())

3103986

Total of 3103986 combinations of ItemCodes

In [27]:
item_id_list = list(item_id_array)
item_id_list = [int(i) for i in item_id_list]
len(item_id_list)

3305

In [30]:
%%time

dict_={}
counting=-1
for item in item_id_list:
    counting+=1
    check_count=0
    dict_[item] = []
    for key, value in count.most_common():
        if item in key:
            list_ = list(key)
            list_.remove(item)
            obj=list_[0]
            dict_[item].append((obj, value))
            check_count+=1
        if check_count>=5 and counting%300==0:
            print(counting)
            print(item, dict_[item])
            break

0
465549 [(465570, 113), (1787079, 112), (1787583, 110), (1732122, 99), (470883, 98)]
300
466032 [(466053, 193), (466011, 179), (477267, 118), (477246, 98), (477288, 90)]
600
340935 [(1787079, 81), (1787583, 72), (445452, 71), (466137, 70), (461853, 68)]
900
481614 [(481593, 42), (477162, 27), (482181, 25), (482160, 23), (477120, 23)]
1200
473403 [(473424, 31), (473319, 26), (472269, 22), (473214, 20), (473487, 20)]
1500
444276 [(469707, 168), (1787079, 121), (999411, 112), (435225, 98), (445452, 91)]
1800
434784 [(434805, 18), (435099, 9), (470883, 9), (482160, 9), (469938, 9)]
2100
451689 [(462609, 132), (451647, 126), (462567, 108), (451668, 98), (1787079, 87)]
2400
492870 [(492849, 27), (463806, 20), (474117, 20), (490224, 19), (493353, 19)]
2700
1782354 [(998886, 8), (1787079, 7), (445452, 7), (442680, 6), (1784538, 6)]
3000
436023 [(436002, 10), (436044, 9), (436065, 9), (436086, 8), (464331, 7)]
3300
747789 [(63, 1), (212835, 1), (315756, 1), (315924, 1), (340977, 1)]
CPU times:

In [0]:
import pickle

with open('answer_dict.pickle', 'wb') as handle:
    pickle.dump(dict_, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('answer_dict.pickle', 'rb') as handle:
    answer_dict = pickle.load(handle)

In [34]:
answer_dict==dict_

True

In [37]:
print(len(answer_dict))

3305


**answer_dict.pickle** contains the answer dictionary. For each ItemCode it returns a list of 5 tuples with an ItemCode and number of times it was bought with the key. See display_function towards the end of notebook for displaying the results.

###Getting labels of ItemCode

In [0]:
df_final2=pd.read_csv('cleaned_data.csv')
df_final2.shape

(534062, 8)

In [0]:
df_final2 = df_final2[df_final2['ItemCode']!=-1]

In [0]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531286 entries, 0 to 534061
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   UserId                  531286 non-null  float64
 1   TransactionId           531286 non-null  int64  
 2   TransactionTime         531286 non-null  object 
 3   ItemCode                531286 non-null  int64  
 4   ItemDescription         531286 non-null  object 
 5   NumberOfItemsPurchased  531286 non-null  int64  
 6   CostPerItem             531286 non-null  float64
 7   Country                 531286 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 36.5+ MB


In [0]:
label_dict={}
checkcount=0
for i in range(df_final2.shape[0]):
    row = df_final2.iloc[i]
    if row['ItemCode'] in label_dict:
        if row['ItemDescription'] not in label_dict[row['ItemCode']]:
            label_dict[row['ItemCode']].append(row['ItemDescription'])
            checkcount+=1
            print(checkcount)
    else:
        label_dict[row['ItemCode']]=[]
        label_dict[row['ItemCode']].append(row['ItemDescription'])

In [0]:
item_id_array = df_final2['ItemCode'].unique()
item_id_array.shape

(3305,)

In [0]:
item_description_array = df_final2['ItemDescription'].unique()
item_description_array.shape

(4030,)

In [0]:
import pickle

with open('label_dict.pickle', 'wb') as handle:
    pickle.dump(label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('label_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [0]:
label_dict==b

True

**label_dict.pickle** contains the dictionary that maps 'ItemCode' to corresponding 'ItemDescriptions'. The 'ItemDescriptions' are given as lists since there are cases of multiple item descriptions for a given item code. See example below.

In [0]:
 b[481635][0]

'BAKING MOULD ROSE MILK CHOCOLATE'

###Answers display

In [0]:
with open('answer_dict.pickle', 'rb') as handle:
    answer_dict = pickle.load(handle)

In [0]:
with open('label_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [0]:
def display_function(itemcode, answer_dict, label_dict):
    print(str(itemcode).center(40,'_'))
    print()
    print(label_dict[itemcode][0].center(40,'_'))
    print()
    print('Frequently bought together items')
    print()
    for i in range(5):
        print(f'\t{answer_dict[itemcode][i][0]}\t{label_dict[answer_dict[itemcode][i][0]][0]}')
        print(f'\t\tBought together {answer_dict[itemcode][i][1]} times')
        print()

Examples are given below:



In [41]:
try:
    display_function(481635, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

_________________481635_________________

____BAKING MOULD ROSE MILK CHOCOLATE____

Frequently bought together items

	481530	BAKING MOULD HEART MILK CHOCOLATE
		Bought together 35 times

	481677	BAKING MOULD CHOCOLATE CUPCAKES
		Bought together 26 times

	482181	JAM MAKING SET PRINTED
		Bought together 24 times

	481551	BAKING MOULD HEART WHITE CHOCOLATE
		Bought together 23 times

	482160	JAM MAKING SET WITH JARS
		Bought together 23 times



In [42]:
try:
    display_function(464226, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

_________________464226_________________

_____MIRROR MOSAIC HURRICANE LAMP ______

Frequently bought together items

	1787079	JUMBO  BAG BAROQUE BLACK WHITE
		Bought together 30 times

	465738	VICTORIAN GLASS HANGING T-LIGHT
		Bought together 29 times

	482181	JAM MAKING SET PRINTED
		Bought together 29 times

	460635	SUKI  SHOULDER BAG
		Bought together 28 times

	447594	SMALL GLASS HEART TRINKET POT
		Bought together 27 times



In [43]:
try:
    display_function(471996, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

_________________471996_________________

____EMPIRE UNION JACK TV DINNER TRAY____

Frequently bought together items

	1787079	JUMBO  BAG BAROQUE BLACK WHITE
		Bought together 54 times

	466137	SMALL POPCORN HOLDER
		Bought together 45 times

	1787583	WHITE HANGING HEART T-LIGHT HOLDER
		Bought together 43 times

	470883	REGENCY CAKESTAND 3 TIER
		Bought together 41 times

	475986	RECIPE BOX PANTRY YELLOW DESIGN
		Bought together 41 times



In [44]:
try:
    display_function(1787079, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

________________1787079_________________

_____JUMBO  BAG BAROQUE BLACK WHITE_____

Frequently bought together items

	470106	JUMBO BAG PINK POLKADOT
		Bought together 967 times

	460551	JUMBO STORAGE BAG SUKI
		Bought together 876 times

	470631	JUMBO SHOPPER VINTAGE RED PAISLEY
		Bought together 836 times

	435225	LUNCH BAG RED RETROSPOT
		Bought together 706 times

	487263	JUMBO BAG VINTAGE DOILY 
		Bought together 705 times



In [45]:
try:
    display_function(465549, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

_________________465549_________________

____FAMILY ALBUM WHITE PICTURE FRAME____

Frequently bought together items

	465570	PICTURE FRAME WOOD TRIPLE PORTRAIT
		Bought together 113 times

	1787079	JUMBO  BAG BAROQUE BLACK WHITE
		Bought together 112 times

	1787583	WHITE HANGING HEART T-LIGHT HOLDER
		Bought together 110 times

	1732122	WOODEN PICTURE FRAME WHITE FINISH
		Bought together 99 times

	470883	REGENCY CAKESTAND 3 TIER
		Bought together 98 times



In [46]:
try:
    display_function(42, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

___________________42___________________

________________CARRIAGE________________

Frequently bought together items

	470883	REGENCY CAKESTAND 3 TIER
		Bought together 21 times

	1787583	WHITE HANGING HEART T-LIGHT HOLDER
		Bought together 18 times

	457590	VINTAGE SNAP CARDS
		Bought together 16 times

	482160	JAM MAKING SET WITH JARS
		Bought together 16 times

	478086	PHOTO FRAME CORNICE
		Bought together 15 times



In [47]:
try:
    display_function(-1, answer_dict=answer_dict, label_dict=b)
except:
    print('*****Invalid ItemId*****')

___________________-1___________________

*****Invalid ItemId*****


**answer_dict.pickle** <br/>
contains final answers dictionary.